<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/python_refresher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
x = 2
y = 3
x + y

5

In [0]:
if x > y:
  print("greater")
else:
  print("not greater")

not greater


In [0]:
for i in range(5):
  print(i, end=',')

0,1,2,3,4,

In [0]:
L = [2,3,5,7,11,13,17]
L

[2, 3, 5, 7, 11, 13, 17]

In [0]:
for prime in L:
  print(prime, end=',')

2,3,5,7,11,13,17,

In [0]:
p = 5
if p in L:
  print(p, ' is prime')

5  is prime


In [0]:
L_extended = L + [19, 23]
L_extended

[2, 3, 5, 7, 11, 13, 17, 19, 23]

In [0]:
L_extended[2:5]

[5, 7, 11]

In [0]:
L_extended[:-3]

[2, 3, 5, 7, 11, 13]

In [0]:
while L:
  L = L[1:]
  print(L)

[3, 5, 7, 11, 13, 17]
[5, 7, 11, 13, 17]
[7, 11, 13, 17]
[11, 13, 17]
[13, 17]
[17]
[]


In [0]:
def func(arg1, arg2):
  return arg1 + arg2

x = func(3, 4)
x

7

In [0]:
[i**2 for i in range(5)]

[0, 1, 4, 9, 16]

In [0]:
class SomeClass:

  def __init__(self, arg1, arg2):
    self._sum = arg1 + arg2
    self._diff = arg1 - arg2

  def printer(self):
    print(self._sum, self._diff)


obj = SomeClass(3, 4)
obj.printer()

7 -1


In [0]:
class SomeOtherClass:

  def __init__(self, arg1, arg2):
    self._sum = arg1 + arg2
    self._diff = arg1 - arg2

  def update_sum(self, arg):
    self._sum = arg

  def printer(self):
    print(self._sum, self._diff)


obj = SomeOtherClass(3, 4)
obj.printer()
obj.update_sum(77)
obj.printer()

7 -1
77 -1
